In [1]:
import os
import pandas as pd
import numpy as np
import json
import re
import duckdb

In [2]:
# Set working directory
work_dir = os.path.join(os.getcwd(), "../")
os.chdir(work_dir)

In [3]:
# Reading in restaurants and cleaned user reviews datasets
restaurants_df = pd.read_csv('data/yelp_restaurants_Phila_cleaned.csv')
user_reviews_df = pd.read_csv('data/yelp_reviews_users_Phila_cleaned.csv')

In [4]:
# Dropping columns that we don't plan on using for our analysis
restaurants_df_clean = restaurants_df.drop(columns = ['BusinessAcceptsCreditCards', 'RestaurantsDelivery', 'RestaurantsTakeOut', 'BusinessParking', \
                                          'OutdoorSeating', 'WiFi', 'BikeParking', 'Caters', 'HasTV'])
user_reviews_df_clean_fin = user_reviews_df.drop(columns = ['user_elite_years', 'useful_restaurant_review_count'])

In [5]:
print("restaurant_reviews_df columns:", restaurants_df.columns)
print("user_reviews_df columns:", user_reviews_df.columns)

restaurant_reviews_df columns: Index(['business_id', 'restaurant_name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'restaurant_review_count',
       'is_open', 'categories', 'BusinessAcceptsCreditCards',
       'RestaurantsPriceRange2', 'RestaurantsDelivery', 'RestaurantsTakeOut',
       'BusinessParking', 'OutdoorSeating', 'WiFi', 'BikeParking',
       'RestaurantsGoodForGroups', 'RestaurantsReservations', 'NoiseLevel',
       'Caters', 'HasTV', 'Alcohol', 'Ambience', 'GoodForKids',
       'RestaurantsAttire', 'GoodForMeal', 'Monday_is_open', 'Tuesday_is_open',
       'Wednesday_is_open', 'Thursday_is_open', 'Friday_is_open',
       'Saturday_is_open', 'Sunday_is_open', 'hours'],
      dtype='object')
user_reviews_df columns: Index(['business_id', 'restaurant_name', 'review_id', 'user_id', 'user_rating',
       'useful_restaurant_review_count', 'review', 'review_date', 'user_name',
       'user_review_count', 'user_average_rating', 'useful_user_review_c

In [6]:
restaurants_df_clean.dtypes

business_id                  object
restaurant_name              object
address                      object
city                         object
state                        object
postal_code                   int64
latitude                    float64
longitude                   float64
restaurant_review_count       int64
is_open                       int64
categories                   object
RestaurantsPriceRange2        int64
RestaurantsGoodForGroups     object
RestaurantsReservations      object
NoiseLevel                   object
Alcohol                      object
Ambience                     object
GoodForKids                  object
RestaurantsAttire            object
GoodForMeal                  object
Monday_is_open                int64
Tuesday_is_open               int64
Wednesday_is_open             int64
Thursday_is_open              int64
Friday_is_open                int64
Saturday_is_open              int64
Sunday_is_open                int64
hours                       

In [7]:
# Cleaning NoiseLevel, Alcohol, and Ambience columns by removing strings that start with u' and single quotes
restaurants_df_clean['NoiseLevel'] = restaurants_df_clean['NoiseLevel'].astype(str).apply(lambda x: re.sub(r"^u'|'", '', x))
restaurants_df_clean['Alcohol'] = restaurants_df_clean['Alcohol'].astype(str).apply(lambda x: re.sub(r"^u'|'", '', x))
restaurants_df_clean['RestaurantsAttire'] = restaurants_df_clean['RestaurantsAttire'].astype(str).apply(lambda x: re.sub(r"^u'|'", '', x))

In [8]:
# Checking unique values in NoiseLevel column
restaurants_df_clean['NoiseLevel'].value_counts()

NoiseLevel
average      166
loud          19
quiet         11
nan            3
very_loud      1
Name: count, dtype: int64

In [9]:
# Checking unique values in Alcohol column
restaurants_df_clean['Alcohol'].value_counts()

Alcohol
none             99
full_bar         87
beer_and_wine    10
nan               4
Name: count, dtype: int64

In [10]:
# Checking unique values in RestaurantsAttire column
restaurants_df_clean['RestaurantsAttire'].value_counts()

RestaurantsAttire
casual    190
dressy      5
nan         5
Name: count, dtype: int64

In [11]:
## Cleaning Ambience and GoodForMeal column to prepare for unnesting
# Function to convert a string to JSON formatted dictionary
def convert_to_dict(string):
    # Check if the string is empty
    if not string:
        return {}

    # Replace u' with '
    json_str = string.replace("u'", "'")

    # Replace single quotes with double quotes and None with null
    json_str = json_str.replace("'", '"').replace("None", "null")
    
    # Ensuring True and False values are in proper JSON format
    json_str = re.sub(r'\bTrue\b', 'true', json_str)  # Convert True to true
    json_str = re.sub(r'\bFalse\b', 'false', json_str)  # Convert False to false

    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        return {}

# Apply the conversion
restaurants_df_clean['Ambience'] = restaurants_df_clean['Ambience'].astype(str).apply(convert_to_dict)
restaurants_df_clean['GoodForMeal'] = restaurants_df_clean['GoodForMeal'].astype(str).apply(convert_to_dict)

In [12]:
# Unnesting Ambience and GoodForMeal column
restaurants_df_clean_ambience = pd.json_normalize(restaurants_df_clean['Ambience'])
restaurants_df_clean_GoodForMeal = pd.json_normalize(restaurants_df_clean['GoodForMeal'])

In [13]:
# Observing columns after unnesting Ambience
print(restaurants_df_clean_ambience)

# Observing unique value counts for each column
print(restaurants_df_clean_ambience.apply(lambda x: x.value_counts(dropna = False)))

    romantic intimate touristy hipster  divey classy trendy upscale casual
0      False    False    False   False  False   True  False   False  False
1      False    False    False   False  False   True  False   False   True
2      False    False     None   False  False  False  False   False   True
3      False    False    False   False  False   True   True   False  False
4      False    False    False   False  False   True  False   False  False
..       ...      ...      ...     ...    ...    ...    ...     ...    ...
195    False    False    False   False   None  False  False   False   True
196    False    False    False   False  False   True  False   False  False
197     True     True    False   False  False   True   None   False   True
198    False    False    False   False  False  False  False   False   True
199     None     None     None    None  False   True   None    None   True

[200 rows x 9 columns]
       romantic  intimate  touristy  hipster  divey  classy  trendy  upscale

In [14]:
# Observing columns after unnesting GoodForMeal
print(restaurants_df_clean_GoodForMeal)

# Observing unique value counts for each column
print(restaurants_df_clean_GoodForMeal.apply(lambda x: x.value_counts(dropna = False)))

    dessert latenight  lunch dinner brunch breakfast
0      None     False   True   True   True      None
1     False     False   None  False   True      True
2     False     False   True   None  False     False
3      None     False  False   True  False     False
4      None     False  False   True  False     False
..      ...       ...    ...    ...    ...       ...
195   False     False   None   True  False     False
196    True      None   None   True   None      None
197    None      None   None   True   None      None
198   False     False  False   True  False     False
199   False     False  False  False  False     False

[200 rows x 6 columns]
       dessert  latenight  lunch  dinner  brunch  breakfast
False      151        126     40      33     119        144
None        31         43     47      13      45         25
True        10         23    105     146      28         23
NaN          8          8      8       8       8          8


In [15]:
# Replacing None values into NA values
restaurants_df_clean_ambience = restaurants_df_clean_ambience.replace({None: np.nan})
restaurants_df_clean_GoodForMeal = restaurants_df_clean_GoodForMeal.replace({None: np.nan})

# Convert all columns to boolean type
restaurants_df_clean_ambience = restaurants_df_clean_ambience.astype('boolean')
restaurants_df_clean_GoodForMeal = restaurants_df_clean_GoodForMeal.astype('boolean')

In [16]:
# merging restaurants_df_clean, restaurants_df_clean_ambience, restaurants_df_clean_GoodForMeal
restaurants_df_clean_fin = pd.concat([restaurants_df_clean, restaurants_df_clean_ambience], axis = 1).drop(columns = 'Ambience')
restaurants_df_clean_fin = pd.concat([restaurants_df_clean_fin, restaurants_df_clean_GoodForMeal], axis = 1).drop(columns = 'GoodForMeal')

In [17]:
# Converting other boolean-like columns to boolean type
restaurants_df_clean_fin['RestaurantsGoodForGroups'] = restaurants_df_clean_fin['RestaurantsGoodForGroups'].astype('boolean')
restaurants_df_clean_fin['RestaurantsReservations'] = restaurants_df_clean_fin['RestaurantsReservations'].astype('boolean')
restaurants_df_clean_fin['GoodForKids'] = restaurants_df_clean_fin['GoodForKids'].astype('boolean')

In [18]:
# Adding back in stars column to the restaurants_df_clean_fin as it was accidentally not included
# Create DuckDB connection
con = duckdb.connect()

# Loading business data
con.sql("CREATE TABLE business AS SELECT * FROM read_json_auto('yelp_data/yelp_academic_dataset_business.json')")
stars_df = con.sql("SELECT business_id, stars FROM business").fetchdf()

# grabbing all business_ids that are in restaurants_df_clean_fin
stars_df = stars_df[stars_df['business_id'].isin(restaurants_df_clean_fin['business_id'])].rename(columns = {'stars': 'restaurant_rating'})

# merging stars_df into restaurants_df_clean_fin
restaurants_df_clean_fin = pd.merge(restaurants_df_clean_fin, stars_df, how = 'left', on = 'business_id')


In [19]:
restaurants_df_clean_fin.columns

Index(['business_id', 'restaurant_name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'restaurant_review_count',
       'is_open', 'categories', 'RestaurantsPriceRange2',
       'RestaurantsGoodForGroups', 'RestaurantsReservations', 'NoiseLevel',
       'Alcohol', 'GoodForKids', 'RestaurantsAttire', 'Monday_is_open',
       'Tuesday_is_open', 'Wednesday_is_open', 'Thursday_is_open',
       'Friday_is_open', 'Saturday_is_open', 'Sunday_is_open', 'hours',
       'romantic', 'intimate', 'touristy', 'hipster', 'divey', 'classy',
       'trendy', 'upscale', 'casual', 'dessert', 'latenight', 'lunch',
       'dinner', 'brunch', 'breakfast', 'restaurant_rating'],
      dtype='object')

In [20]:
# Writing out fully cleaned final datasets
restaurants_df_clean_fin.to_csv('data/yelp_restaurants_Phila_final.csv', index = False)
user_reviews_df_clean_fin.to_csv('data/yelp_reviews_users_Phila_final.csv', index = False)